# Project: Zero-Knowledge Password Verification (Schnorr Protocol)

### **What is this?**
This project demonstrates a **Zero-Knowledge Proof (ZKP)** authentication system. It allows a user (Client) to prove they know a password to a server (Verifier) **without actually sending the password**.

### **The Mathematical Logic (Schnorr Protocol)**
We use large numbers and discrete logarithms. 
* **$x$**: The Secret (Password).
* **$y$**: The Public Key ($y = g^x \pmod p$).
* **$r$**: A random number generated by the user for one session.

**The Interaction:**
1. **Commitment ($t$):** Client calculates $t = g^r$ and sends it to Server.
2. **Challenge ($c$):** Server picks a random number $c$ and sends it to Client.
3. **Response ($s$):** Client calculates $s = r + c \cdot x$ and sends it to Server.
4. **Verification:** Server checks if $g^s == t \cdot y^c$. If true, the Client must know $x$.

In [ ]:
import random
import hashlib

# --- 1. GLOBAL PARAMETERS ---
# In a real app, P would be a massive prime number (256 bits).
# We use a smaller prime here so the math is easy to inspect.
P = 48611  # A prime number
G = 19     # A generator

# --- 2. HELPER FUNCTION ---
def hash_password_to_int(password_str):
    """
    Converts a text password (e.g., 'secret123') into a mathematical integer.
    This acts as our secret 'x'.
    """
    # SHA256 Hash the string
    sha_signature = hashlib.sha256(password_str.encode()).hexdigest()
    # Convert Hex to Integer
    big_int = int(sha_signature, 16)
    # Modulo (P-1) to fit our field
    secret_int = big_int % (P - 1)
    return secret_int

# --- 3. THE CLIENT (Prover) ---
class Prover:
    def __init__(self, password_str):
        # The client knows the secret 'x'
        self.secret_x = hash_password_to_int(password_str)
        self.r = 0

    def register(self):
        # Calculate Public Key: y = G^x mod P
        public_y = pow(G, self.secret_x, P)
        return public_y

    def generate_commitment(self):
        # Step 1: Pick random 'r' and compute 't'
        self.r = random.randint(1, P - 2)
        t = pow(G, self.r, P)
        return t

    def solve_challenge(self, c):
        # Step 3: Compute response 's = r + c * x'
        s = self.r + (c * self.secret_x)
        return s

# --- 4. THE HACKER (Fake Prover) ---
class FakeProver:
    def __init__(self):
        # Hacker DOES NOT know the secret 'x'
        self.r = 0

    def generate_commitment(self):
        # Hacker can still generate a valid random commitment
        self.r = random.randint(1, P - 2)
        t = pow(G, self.r, P)
        return t

    def solve_challenge(self, c):
        # Hacker attempts to guess 'x'. Let's assume they guess 12345.
        # Since this guess is likely wrong, the proof will fail.
        fake_x = 12345 
        s = self.r + (c * fake_x)
        return s

# --- 5. THE SERVER (Verifier) ---
class Verifier:
    def __init__(self, public_key):
        # Server ONLY knows Public Key 'y'
        self.public_y = public_key
        self.c = 0

    def generate_challenge(self):
        # Step 2: Server picks random challenge 'c'
        self.c = random.randint(1, 100)
        return self.c

    def verify(self, t, s):
        # Step 4: Verification Equation
        # Check if: G^s == t * y^c
        
        lhs = pow(G, s, P)
        rhs = (t * pow(self.public_y, self.c, P)) % P
        
        print(f"   [Debug] LHS (G^s): {lhs}")
        print(f"   [Debug] RHS (t*y^c): {rhs}")
        return lhs == rhs

# --- 6. RUN SIMULATION ---
def run_zkp_system():
    print("--- SETUP PHASE ---")
    password_input = input("Enter a password to register: ")
    
    # User registers
    peggy = Prover(password_input)
    public_key = peggy.register()
    
    # Server saves public key
    victor = Verifier(public_key)
    print(f"Registered! Public Key (y): {public_key}")
    print("Server does NOT know the password.")
    
    while True:
        print("\n--- LOGIN MENU ---")
        print("1. Login as User (Peggy)")
        print("2. Login as Hacker (Mallory)")
        print("3. Exit")
        choice = input("Selection: ")
        
        if choice == '3': 
            break
        
        # Select who is attempting to login
        if choice == '1':
            current_actor = peggy
            print("\n>> Legitimate User Attempting Login...")
        else:
            current_actor = FakeProver()
            print("\n>> Hacker Attempting Login...")
            
        # --- THE PROTOCOL FLOW ---
        
        # 1. Commitment
        t = current_actor.generate_commitment()
        print(f"1. Actor sends Commitment (t): {t}")
        
        # 2. Challenge
        c = victor.generate_challenge()
        print(f"2. Server sends Challenge (c): {c}")
        
        # 3. Response
        s = current_actor.solve_challenge(c)
        print(f"3. Actor sends Response (s): {s}")
        
        # 4. Verification
        result = victor.verify(t, s)
        
        if result:
            print("SUCCESS: Identity Verified! Access Granted.")
        else:
            print("FAILURE: Math does not match. Access Denied.")

# Run the app
run_zkp_system()

### **Code Explanation**

**1. Global Parameters**
We define a prime number `P` and a generator `G`. These define the "mathematical playground" (Finite Field) where our calculations happen. In cryptography, these numbers are public and standard.

**2. Hashing (`hash_password_to_int`)**
We cannot do math on the string "Password123". We use SHA256 to turn the text into a huge number, then shrink it down to fit in our field. This is our secret `x`.

**3. The Prover Class (Peggy)**
* `register()`: Calculates the public key `y`. This is safe to share.
* `solve_challenge()`: Uses the secret `x` to answer the server's question. This is the only time `x` is used, but `x` is never sent over the network.

**4. The FakeProver Class (Mallory)**
Mallory tries to log in. She can generate a random commitment `t` just fine. But when she receives challenge `c`, she is stuck. She needs to calculate `s = r + c * x`. Since she doesn't know `x`, she guesses. The math will fail.

**5. The Verifier Class (Victor/Server)**
Victor only knows `y`. He checks the proof using `G^s == t * y^c`. This equation will ONLY balance if the `s` was calculated using the correct `x`.